# Foundations of Computer Science (1 of 4)


Fill in the various `TODO` items, then submit a copy of this notebook 
to Craig Ferguson (`me@craigfe.io`) at least 48 hours before the supervision. 

Do _not_ use any helper modules in the standard library (`List`, `Set`,
`Hashtbl` etc.). In particular, you should define utilities such as `filter`,
`map` and `fold` etc. yourself.


In [1]:
let crsid = "gtf23"

;; assert (crsid <> "TODO")

val crsid : string = "TODO"


error: runtime_error

<hr/>

## 1. Type inference

Briefly justify the inferred types of each of the following values:


In [ ]:
let a x y = x

x and y can be of any type independently, the return type is the same as x's type.

In [ ]:
let b x y z = if x then y else z

x must be bool because of the if, and since y and z can both be returned, their type, and the return type all are the same.

In [ ]:
let c x y = x y

The return is x applied to y, so x is a function of any type, and y is of the input type of x.

In [ ]:
let c' x y = x (y : _ -> _)

y is declared to be some function, so x is a function mapping from some function to some type.

In [ ]:
let rec d _ x = d [ x ] true

d is called on [x] and true, so x is bool, so the first argument is a bool list.

Consider the following questions, which we may discuss in the supervision:

- What _is_ a type?
- What does it mean for two types to be _equal_?
- Can an OCaml expression have no type?
- Can an OCaml expression have more than one type?


<hr/>

## 2. Domain modelling

> Relevant reading: Chapter 6 of the notes

In statically-typed programming, the first step of problem-solving is to consider the internal shape of the data being manipulated and (if necessary) define types that capture that structure, ideally in a way that makes invalid states inexpressible. If our types are too large – that is, contain invalid terms – our code will need to assert that fact at runtime.

For instance, if we're writing a banking system in which each transaction contains one-or-more participants, using a `participant list` to encode this relationship admits the invalid state `[]`:


```ocaml
type participants = participant list

(* .. *)
let process_transaction : participants -> response = function
  | initiator :: others -> handle_transaction initiator others
  | [] -> assert false (* This can never happen in practice, I promise.
                          Trust me, I'm a professional. *)
```

This relationship betwen types (correctness via proof) and run-time assertions (correctness via pinky-promise) is one of the reasons that it's important to pick the right types for your problem. Even more importantly, it's an easy source of introductory exam questions, so worth focussing on.

For each of the following sets (or families of sets), give a type (or sequence of types) that models it:

- The complex numbers.

In [ ]:
type complex = float*float

- The RGB colour space.

In [ ]:
type RGB = int*int*int (* Q: How could we limit it to 0-255? *)

- For each set $S$, all permutations of zero-or-more elements of $S$.

In [ ]:
type 'a permutation = 'a list;

- For each set $S$, the set $S \cup \{ \texttt{Bottom} \}$ containing a _new_ constant term $\texttt{Bottom}$ that is not in $S$.

In [ ]:
type 'element permutationWithBottom =
    Bottom
    | 'element list;

> **Hint**: the polymorphic type `'a t` can be considered as a _family_ of types, indexed by a particular choice of `'a`. For instance, `'a list` is a family including the types `int list` and `string list`.

Consider the following questions, which we may discuss in the supervision:

- Are there any OCaml types $t$ such that there are no expressions of type $t$?

- Are there any OCaml types $t$ such that there are no _values_ of type $t$ (i.e. empty types)? Would such a type have any practical meaning?

<hr/>

## 3. Mystery complexities

> Relevant reading: Chapter 2 of the notes


For each of the following functions:

- Give a concise description of their _behaviour_ (not their implementation) – 1 mark each
- State their space and time complexities _in terms of their non-functional parameters_, being as precise in your statement as possible – 2 marks each

Assume that the compiler performs no optimisations on the given code.


In [2]:
let rec f pred l =
  match l with
  | [] -> 0
  | x :: xs -> if pred x then 1 else 0 + f pred xs

val f : ('a -> bool) -> 'a list -> int = <fun>


In [ ]:
pred is a bool function with some input type 'a. 
Given pred and an 'a list, f counts the number of elements in l for which pred is true.
Time complexity is linear with list length, and so is space complexity. (It is not a tail recursive function.)

In [ ]:
let f' pred l =
  let rec inner l acc =
    match l with
    | [] -> acc
    | x :: xs ->
        let inc = if pred x then 1 else 0 in
        inner xs (acc + inc)
  in
  inner l 0

In [ ]:
This is the same as above, except that it is tail-recursive, so space complexity is constant.

In [ ]:
let rec g n = if n > 1 then g (n - 1) + g (n - 2) else n

In [ ]:
This is the Fibonacci function. There are always two function calls for (n-1) and (n-2), this makes the time and space complexity exponential.

In [10]:
let rec h l =
  let rec id_map xs =
    match xs with
    | [] -> []
    | x :: xs -> x :: id_map xs
  in
  match l with
  | [] -> []
  | _ :: tl -> id_map l :: h tl

val h : 'a list -> 'a list list = <fun>


In [9]:
let l = 1::2::[3];;
h l

val l : int list = [1; 2; 3]


- : int list list = [[1; 2; 3]; [2; 3]; [3]]


In [ ]:
id_map is an identity mapping - for any input list, it returns the same list. 
However, while doing so, it takes up space and uses time in linear proportion to the lenght of its input.
So, function h generates a list of lists, where the first list is the input list, the next is the input list but its first element,
the second is the input list with the first two elements omitted and so on until a single element list with the input list's last element in it.
The time complexity is n^2, because at the ith recursion we go over n-i elements. 
The space complexity is the same, because we are holding in memory all the subresults. (The sublists.)

Consider the following questions, which we may discuss in the supervision:

- What is the time complexity of integer addition?
- Are there any programs with time complexity $t(n)$ (for some $n$) and space complexity $s(n)$ such that $s$ grows more quickly than $t$?

<hr/>

## 4. Factors


Write a function `factors` that converts a list of integers into a
list of lists of the factors of those integers (each in increasing order).
For example,
      
`factors [1; 4; 12] = [[1]; [1; 2; 4]; [1; 2; 3; 4; 6; 12]]`


In [33]:
let rec divisors x y =
    if (y = x) then [x]
    else 
        if (x mod y) = 0 then y::divisors x (y+1)
        else divisors x (y+1)
    
let rec factors list = 
match list with
    |[] -> []
    |x::xs -> (divisors x 1)::factors xs

val divisors : int -> int -> int list = <fun>


val factors : int list -> int list list = <fun>


In [34]:
(* Tests of [factors] (feel free to add your own!) *)

;; assert (factors [] = [])
;; assert (factors [1] = [[1]])

- : unit = ()


- : unit = ()


## 5. Sets (1995 Paper 1 Question 3)

> Relevant reading: Chapter 8 of the notes

An OCaml list can be considered to be a set if it has no repeated elements,
e.g., `[4; 2; 3]` is a set but `[4; 2; 4; 3]` is not.

Write an OCaml function `intersect` to compute the set-theoretic intersection of
two lists that satisfy this property of being a set. (The intersection of two
sets is the set of elements that appear in both sets.) Your function must also
satisfy conditions (1)–(3) below:

1. The result list has no repeated elements;

2. The number of cons (`::`) operations performed does not exceed the number of
elements in the result list.

3. The elements of the result list appear in the same order as they do in the
first argument list.

You may assume the existence of `[]` (nil) and `::` (cons). All other functions over
lists must be defined by you. Little credit will be given for answers that do
not satisfy conditions (1)–(3).


In [44]:
let rec contains l v =
    match l with 
    |[] -> false
    |x::xs -> 
    if x = v then true else contains xs v
    
let rec intersect list1 list2 =
    match list1 with
    |[]->[]
    |x::xs -> 
        if contains list2 x then x::intersect xs list2
        else intersect xs list2

val contains : 'a list -> 'a -> bool = <fun>


val intersect : 'a list -> 'a list -> 'a list = <fun>


In [45]:
(* Tests of [intersect] *)

;; assert (intersect [] [1; 2; 3] = [])
;; assert (intersect [ 1 ] [ 1 ] = [ 1 ])

- : unit = ()


- : unit = ()



Write an OCaml function `subtract` that given two lists satisfying the
property of being a set, returns a list consisting of those elements of
the first list that do not appear in the second list. Your subtract
function must satisfy conditions (a)–(c) above.


In [52]:
let rec subtract list1 list2 =
    match list1 with
    |[]->[]
    |x::xs -> 
        if contains list2 x then intersect xs list2
        else x::intersect xs list2

val subtract : 'a list -> 'a list -> 'a list = <fun>


In [53]:
(* Tests of [subtract] *)

;; assert (subtract [1] [1] = [])

- : unit = ()




Write an OCaml function `union` that given two lists satisfying the
property of being a set, returns a list consisting of those elements that appear
in one or other or both of the lists. Your union function must satisfy
conditions (1)–(3) above and (4) below:
      
4. Elements from the first argument list appear before any others in the result.


In [55]:
let rec append list1 list2 =
    match list1 with
    |[] -> list2
    |x::xs -> x::(append xs list2)

let union list1 list2 =
    append list1 (subtract list2 list1)

val append : 'a list -> 'a list -> 'a list = <fun>


val union : 'a list -> 'a list -> 'a list = <fun>


In [56]:
(* Tests of [union] *)

;; assert (union [1] [2] = [1; 2])

- : unit = ()
